# 🎯Project Trident

### 🔗Ένωση Πινάκων

In [1]:
from httpx import head
import pandas as pd

# Φόρτωση δεδομένων από αρχεία CSV
df_std = pd.read_csv('standard_stats.csv')
df_shoot = pd.read_csv('shooting.csv')

# Καθαρισμός στηλών απο επικεφαλίδες που επαναλαμβάνονται
df_std = df_std[df_std['Rk'] != 'Rk']
df_shoot = df_shoot[df_shoot['Rk'] != 'Rk']

# Epιλογή συγκεκριμένων στηλών για διατήρηση απο το dataframe df_shoot
shooting_cols = [
    'Player', 'Squad',
    'Sh', # Σουτ συνολικά (Shots)
    'SoT', # Σουτ στην εστία (Shots on Target)
    'SoT%', # Ποσοστό σουτ στην εστία (Shots on Target %)
    'Sh/90', # Σουτ ανά 90 λεπτά (Shots per 90)
    'SoT/90', # Σουτ στην εστία ανά 90 λεπτά (Shots on Target per 90)
    'G/Sh', # Γκολ ανά σουτ (Goals per Shot)
    'G/SoT' # Γκολ ανά σουτ στην εστία (Goals per Shot on Target)
]

# Διατήρηση μόνο των στηλων που υπάρχουν στο αρχειο df_shoot
available_cols = [c for c in shooting_cols if c in df_shoot.columns]
df_shoot_clean = df_shoot[available_cols].copy()

# Μεττροποίηση αριθμητικών στηλών
for col in available_cols[2:]:  # Ξεκινάμε από την τρίτη στήλη
    df_shoot_clean[col] = pd.to_numeric(df_shoot_clean[col], errors='coerce').fillna(0)
    
    
# Συγχώνευση των δύο dataframes με βάση τις στήλες 'Player' και 'Squad' σαν ID κλειδιά
df = pd.merge(df_std, df_shoot_clean, on=['Player', 'Squad'], how='left')

# Γεμισμα κενών τιμών με 0 για τις αριθμητικές στήλες απο το df_shoot
df = df.fillna({col: 0 for col in available_cols[2:]})

# Διορθωση των βασικών στηλών του Standard dataframe
df['Min'] = df['Min'].astype(str).str.replace(',', '', regex=False).astype(float)
df['Age'] = df['Age'].astype(str).str.split('-').str[0] # Κρατάμε μόνο το ακέραιο μέρος από την ηλικία
df['Age'] = pd.to_numeric(df['Age'], errors='coerce').fillna(0).astype(int)

# Μετατροπή στηλων Gls και Ast κλπ σε αριθμητικές τιμές
std_metrics = ['Gls', 'Ast', 'G-PK', 'PK', 'PKatt']
for col in std_metrics:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
        
# Reset index για καθαρη αρίθμηση
df = df.reset_index(drop=True)

# Αποθήκευση του τελικού dataframe σε νέο αρχείο CSV
filename = 'perfect_merge.csv'
df.to_csv(filename, index=False)

print("✅ Perfect Merge")
print(f"Σύνολο παικτών μετά τη συγχώνευση: {len(df)}")
print(f"Σύνολο στηλών μετά τη συγχώνευση: {len(df.columns)}")

# Εμφάνιση τοπ 5 γραμμών του τελικού dataframe
df.head()

✅ Perfect Merge
Σύνολο παικτών μετά τη συγχώνευση: 2523
Σύνολο στηλών μετά τη συγχώνευση: 32


,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,...,G+A.1,G-PK.1,G+A-PK,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT
0,1,Brenden Aaronson,us USA,"FW,MF",Leeds United,eng Premier League,25,2000,22,16,...,0.44,0.25,0.44,30,14,46.7,1.90,0.89,0.13,0.29
1,2,Zach Abbott,eng ENG,DF,Nottingham Forest,eng Premier League,19,2006,1,0,...,0.00,0.00,0.00,0,0,0.0,0.00,0.00,0.00,0.00
2,3,Jones El-Abdellaoui,ma MAR,"MF,FW",Celta Vigo,es La Liga,20,2006,12,2,...,0.48,0.48,0.48,12,3,25.0,2.86,0.71,0.17,0.67
3,4,Himad Abdelli,dz ALG,MF,Angers,fr Ligue 1,26,1999,13,11,...,0.19,0.00,0.00,11,5,45.5,1.05,0.48,0.00,0.00
4,5,Ali Abdi,tn TUN,"MF,DF",Nice,fr Ligue 1,32,1993,11,7,...,0.00,0.00,0.00,0,0,0.0,0.00,0.00,0.00,0.00


### ⚔️ Διαχωρισμός Ρόλων

In [2]:
import pandas as pd

# Φόρτωση δεδομένων απο το τελικό αρχείο CSV
df = pd.read_csv('perfect_merge.csv')

# Κρατάμε μόνο τους επιθετικούς παίκτες
df = df[df['Pos'].str.contains('FW|MF', na=False)].copy()

# Αποκλεισμός των αμυντικών μέσων (Defensive Midfielders)
df['Sh/90'] = pd.to_numeric(df['Sh/90'], errors='coerce').fillna(0)
df = df[(df['Pos'].str.contains('FW', na=False)) | 
        ((df['Pos'].str.contains('MF', na=False)) & (df['Sh/90'] >= 1))].copy()

# Βάσει του Glossary απο το fbref.com, χρειαζόμαστε: Sh/90, G/Sh, SoT%, Ast/90

# Υπολοωισμος των ασιστ ανά 90 λεπτά
df['Ast_per_90'] = (df['Ast'] / df['Min']) * 90

# Σιγουρεμα οτι τα νουμερα είναι καθαρά (fill NaN με 0)
colls_needed = ['Sh/90', 'G/Sh', 'SoT%', 'Ast_per_90']
for col in colls_needed:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
    else:
        df[col] = 0

# Διαχωρισμοός των ρόλων αναλογικά με τις θέσεις που παίζουν οι παίκτες
def get_role(row):
    
    pos = str(row['Pos'])
    shots = row['Sh/90']
    efficiency = row['G/Sh']
    shot_accuracy = row['SoT%']
    assists = row['Ast_per_90']
    goals = row['Gls']
    
    # ΚΑΤΗΓΟΡΙΑ 1: ΕΞΤΡΕΜ + ΔΗΜΙΟΥΡΓΟΙ (FW + MF)
    
    if 'MF' in pos:
        # Shadow Striker
        if assists >= 0.19 and shot_accuracy >= 30:
            return '👻 Shadow Striker / Creator'
        
        elif shots >= 2.8 and shot_accuracy >= 35:
            return '🚀 Winger / Inside Forward'
        
        elif shots > 2.5:
            return '⚡ Winger (Attacking)'
        
        else:
            return '🏹 Supporting Winger'
        
        
    # ΚΑΤΗΓΟΡΙΑ 2: ΚΕΝΤΡΙΚΟΙ ΕΠΙΘΕΤΙΚΟΙ (FW) ΚΑΘΑΡΑ 9αρια
    else:
        # Killer Striker
        if efficiency >= 0.15 and shot_accuracy > 35:
            return '💀 Killer Striker'
        
        elif shots >= 3.0 and goals >= 5: 
            return '🎯 Elite Striker'
        
        elif shots >= 2.2:
            return '⚽ Striker'
        
        else:
            return '🔗 Support Striker'
        
        
# Εφαρμογή της συνάρτησης για κάθε γραμμή του dataframe
df['Role'] = df.apply(get_role, axis=1)

# Φίλτρο Λεπτών συμμετοχής (> 500 λεπτά)
df_final = df[(df['Min'] > 450)].sort_values(by='Gls', ascending=False)
df_final = df_final.reset_index(drop=True)


# print("=== ΚΑΤΑΝΟΜΗ ΜΕΤΡΙΚΩΝ ===")
# print(df[['Sh/90', 'G/Sh', 'SoT%', 'Ast_per_90']].describe())

# print("\n=== ΚΑΤΑΝΟΜΗ ΡΟΛΩΝ ===")
# print(df_final['Role'].value_counts())

print(f"Σύνολο επιθετικών παικτών μετά το φιλτράρισμα: {len(df_final)}")
df_final[['Player', 'Role', 'Gls', 'G/Sh', 'SoT%']].head(5)
        

Σύνολο επιθετικών παικτών μετά το φιλτράρισμα: 698


,Player,Role,Gls,G/Sh,SoT%
0,Erling Haaland,💀 Killer Striker,20,0.21,50.6
1,Thiago,💀 Killer Striker,16,0.20,57.4
2,Vedat Muriqi,💀 Killer Striker,14,0.17,50.8
3,Mason Greenwood,👻 Shadow Striker / Creator,12,0.13,42.2
4,Deniz Undav,🚀 Winger / Inside Forward,11,0.17,38.1


### 🧠 Similarity Search Engine 

In [3]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import euclidean_distances

search_metrics = ['G-PK', 'Sh/90', 'G/Sh', 'SoT%', 'Ast_per_90']

def select_player(df):

    while True:
        
        query = input("\n🔎 Ποιον παίκτη ψάχνεις; (Γράψε 'exit' για έξοδο): ").strip()
        
        if query.lower() == 'exit':
            print("👋 Έξοδος από το πρόγραμμα.")
            break
        
        # Εύρεση του παίκτη στο dataframe
        matches = df[df['Player'].str.contains(query, case=False, na=False)]
        
        count_matches = len(matches)
        
        # ΠΕΡΙΠΤΩΣΗ 1: ΔΕΝ ΒΡΕΘΗΚΕ ΚΑΝΕΙΣ ❌
        if count_matches == 0:
            print(f"❌ Δεν βρέθηκε κανένας παίκτης με το όνομα '{query}'. Δοκίμασε ξανά.")
            continue
        
        # ΠΕΡΙΠΤΩΣΗ 2: ΒΡΕΘΗΚΑΝ ΠΟΛΛΟΙ 👥
        elif count_matches > 1:
            print(f"👥 Βρέθηκαν {count_matches} παίκτες με το όνομα '{query}'. Ποιον εννοείς;")
            print("-" * 40)
            
            # Εμφάνιση των βρεθέντων παικτών με βασικές πληροφορίες και νουμερο σειράς
            matches = matches.reset_index(drop=True)
            for idx, row in matches.iterrows():
                print(f"{idx}. {row['Player']} | Ομάδα: {row['Squad']} - {row['Age'] } ετών")
                
            print("-" * 40)
            
            try:
                selection = int(input(f"👉 Διάλεξε τον σωστό αριθμό (0-{count_matches-1}): "))
                if 0 <= selection < count_matches:
                    target = matches.iloc[selection]
                    print(f"✅ Επέλεξες: {target['Player']} από την ομάδα {target['Squad']}.")
                    return target
            
                else:
                    print("❌ Μη έγκυρη επιλογή. Δοκίμασε ξανά.")
                    continue
            except ValueError:
                print("❌ Πρέπει να δώσεις αριθμό! Πάμε πάλι.")
            
        else :
            target = matches.iloc[0]
            print(f"✅ Βρέθηκε ο παίκτης: {target['Player']} από την ομάδα {target['Squad']}.")
            return target
        
        
# Υπολογισμός ομοιότητας παικτών
def find_similarity(df, target_player, features, top_n=10):
    
    # Scaling των χαρακτηριστικών
    scaler = MinMaxScaler()
    scaled_features = scaler.fit_transform(df[features])
    
    # Βρίσκουμε τη θέση του παίκτη μας στο DataFrame
    target_index = df[(df['Player'] == target_player['Player']) & (df['Squad'] == target_player['Squad'])].index[0]
    
    # Υπολογισμός αποστάσεων
    distances = euclidean_distances(scaled_features[target_index].reshape(1, -1), scaled_features)
    
    # Δημιουργία Similarity Score
    df_res = df.copy()
    df_res['Similarity'] = (1 / (1 + distances[0])) * 100  # Μετατροπή σε ποσοστό
    
    # Φιλτράρισμα για τον ίδιο παίκτη (αποκλεισμός του εαυτού του)
    results = df_res[df_res.index != target_index].sort_values(by='Similarity', ascending=False)
    
    # Μορφοποίηση του Similarity Score
    final = results.head(top_n).copy()
    final['Similarity_Score'] = final['Similarity'].apply(lambda x: f"{x:.1f}%")
    
    return final[['Player', 'Squad', 'Role', 'Gls', 'G/Sh', 'SoT%', 'Similarity_Score']]

# ΕΚΤΕΛΕΣΗ ΠΡΟΓΡΑΜΜΑΤΟΣ
target_player = select_player(df_final)

if target_player is not None:
    print("\n" + "="*60)
    print(f"🔎 ΑΝΑΛΥΣΗ: {target_player['Player']}")
    print(f"🏷️ Ρόλος: {target_player['Role']}") # Εδώ θα τυπώσει το 💀 Killer Striker αυτόματα!
    print(f"📊 Stats: {target_player['Gls']} Γκολ | {target_player['G/Sh']:.2f} G/Sh")
    print("="*60)
    
    matches = find_similarity(df_final, target_player, search_metrics, top_n=10)
    
    print(f"\n✅ ΟΙ 10 ΚΟΡΥΦΑΙΟΙ ΔΙΑΔΟΧΟΙ:")
    display(matches)

✅ Βρέθηκε ο παίκτης: Erling Haaland από την ομάδα Manchester City.

🔎 ΑΝΑΛΥΣΗ: Erling Haaland
🏷️ Ρόλος: 💀 Killer Striker
📊 Stats: 20 Γκολ | 0.21 G/Sh

✅ ΟΙ 10 ΚΟΡΥΦΑΙΟΙ ΔΙΑΔΟΧΟΙ:


,Player,Squad,Role,Gls,G/Sh,SoT%,Similarity_Score
5,Ferrán Torres,Barcelona,💀 Killer Striker,11,0.26,53.5,68.8%
4,Deniz Undav,Stuttgart,🚀 Winger / Inside Forward,11,0.17,38.1,67.3%
1,Thiago,Brentford,💀 Killer Striker,16,0.20,57.4,65.3%
2,Vedat Muriqi,Mallorca,💀 Killer Striker,14,0.17,50.8,65.3%
9,Robert Lewandowski,Barcelona,💀 Killer Striker,9,0.20,52.5,63.6%
3,Mason Greenwood,Marseille,👻 Shadow Striker / Creator,12,0.13,42.2,63.5%
11,Pavel Šulc,Lyon,⚡ Winger (Attacking),9,0.30,43.3,63.3%
10,Haris Tabakovic,Gladbach,💀 Killer Striker,9,0.24,42.1,63.2%
21,Esteban Lepaul,Rennes,💀 Killer Striker,8,0.18,40.9,62.7%
28,Christian Pulisic,Milan,💀 Killer Striker,8,0.27,56.7,62.6%


### 🔬 Βελτιωμένες Μέθοδοι Similarity

Δοκίμασε τις βελτιωμένες εκδοχές που δίνουν **υψηλότερα και πιο ρεαλιστικά scores**:

**Μέθοδος 1: Cosine Similarity** 
- Μετράει τη γωνία μεταξύ vectors αντί για απόσταση
- Δίνει καλύτερα scores (συνήθως 70-95% για παρόμοιους παίκτες)

**Μέθοδος 2: Βελτιωμένη Euclidean**
- Χρησιμοποιεί exponential decay αντί για 1/(1+d)
- Πιο "μαλακή" στις τιμωρίες για μικρές διαφορές

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

# 🔥 ΜΕΘΟΔΟΣ 1: COSINE SIMILARITY (Προτείνεται!)
def find_similarity_cosine(df, target_player, features, top_n=10):
    """
    Χρησιμοποιεί Cosine Similarity για καλύτερα scores.
    Μετράει τη γωνία μεταξύ vectors αντί για απόσταση.
    """
    # Φίλτρο: Μόνο ίδιος ρόλος για ακόμα καλύτερα αποτελέσματα
    same_role_df = df[df['Role'] == target_player['Role']].copy()
    
    if len(same_role_df) < 2:
        print(f"⚠️ Δεν βρέθηκαν αρκετοί παίκτες με ρόλο '{target_player['Role']}'")
        return pd.DataFrame()
    
    # Scaling των χαρακτηριστικών
    scaler = MinMaxScaler()
    scaled_features = scaler.fit_transform(same_role_df[features])
    
    # Εύρεση του target player
    target_mask = (same_role_df['Player'] == target_player['Player']) & (same_role_df['Squad'] == target_player['Squad'])
    target_index = same_role_df[target_mask].index[0]
    target_position = same_role_df.index.get_loc(target_index)
    
    # Υπολογισμός Cosine Similarity
    similarities = cosine_similarity(
        scaled_features[target_position].reshape(1, -1),
        scaled_features
    )[0]
    
    same_role_df['Similarity'] = similarities * 100  # Ήδη 0-1, απλά x100
    
    # Αποκλεισμός του ίδιου παίκτη
    results = same_role_df[same_role_df.index != target_index].sort_values(
        by='Similarity', ascending=False
    )
    
    # Μορφοποίηση
    final = results.head(top_n).copy()
    final['Similarity_Score'] = final['Similarity'].apply(lambda x: f"{x:.1f}%")
    
    return final[['Player', 'Squad', 'Role', 'Gls', 'G/Sh', 'SoT%', 'Similarity_Score']]


# ΕΚΤΕΛΕΣΗ ΜΕ COSINE SIMILARITY
print("=" * 60)
print("🔥 ΜΕΘΟΔΟΣ 1: COSINE SIMILARITY")
print("=" * 60)

target_player_cosine = select_player(df_final)

if target_player_cosine is not None:
    print("\n" + "="*60)
    print(f"🔎 ΑΝΑΛΥΣΗ: {target_player_cosine['Player']}")
    print(f"🏷️ Ρόλος: {target_player_cosine['Role']}")
    print(f"📊 Stats: {target_player_cosine['Gls']} Γκολ | {target_player_cosine['G/Sh']:.2f} G/Sh")
    print("="*60)
    
    matches_cosine = find_similarity_cosine(df_final, target_player_cosine, search_metrics, top_n=10)
    
    if not matches_cosine.empty:
        print(f"\n✅ ΟΙ 10 ΚΟΡΥΦΑΙΟΙ ΔΙΑΔΟΧΟΙ (Cosine Similarity):")
        display(matches_cosine)

🔥 ΜΕΘΟΔΟΣ 1: COSINE SIMILARITY
✅ Βρέθηκε ο παίκτης: Erling Haaland από την ομάδα Manchester City.

🔎 ΑΝΑΛΥΣΗ: Erling Haaland
🏷️ Ρόλος: 💀 Killer Striker
📊 Stats: 20 Γκολ | 0.21 G/Sh

✅ ΟΙ 10 ΚΟΡΥΦΑΙΟΙ ΔΙΑΔΟΧΟΙ (Cosine Similarity):


,Player,Squad,Role,Gls,G/Sh,SoT%,Similarity_Score
32,Ollie Watkins,Aston Villa,💀 Killer Striker,8,0.16,42.9,98.3%
12,Dominic Calvert-Lewin,Leeds United,💀 Killer Striker,9,0.16,46.5,94.8%
6,Joaquín Panichelli,Strasbourg,💀 Killer Striker,11,0.18,50.0,93.9%
5,Ferrán Torres,Barcelona,💀 Killer Striker,11,0.26,53.5,93.6%
2,Vedat Muriqi,Mallorca,💀 Killer Striker,14,0.17,50.8,93.4%
21,Esteban Lepaul,Rennes,💀 Killer Striker,8,0.18,40.9,92.6%
67,Santiago Castro,Bologna,💀 Killer Striker,6,0.18,39.4,92.6%
10,Haris Tabakovic,Gladbach,💀 Killer Striker,9,0.24,42.1,92.5%
50,Nick Woltemade,Newcastle United,💀 Killer Striker,7,0.21,41.4,90.3%
1,Thiago,Brentford,💀 Killer Striker,16,0.20,57.4,90.2%


In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import euclidean_distances

# 🔥 ΜΕΘΟΔΟΣ 2: ΒΕΛΤΙΩΜΕΝΗ EUCLIDEAN ΜΕ EXPONENTIAL DECAY
def find_similarity_euclidean_improved(df, target_player, features, top_n=10):
    """
    Χρησιμοποιεί βελτιωμένη φόρμουλα με exponential decay.
    Πιο "μαλακή" στις μικρές διαφορές, καλύτερα scores.
    """
    # Φίλτρο: Μόνο ίδιος ρόλος
    same_role_df = df[df['Role'] == target_player['Role']].copy()
    
    if len(same_role_df) < 2:
        print(f"⚠️ Δεν βρέθηκαν αρκετοί παίκτες με ρόλο '{target_player['Role']}'")
        return pd.DataFrame()
    
    # Scaling των χαρακτηριστικών
    scaler = MinMaxScaler()
    scaled_features = scaler.fit_transform(same_role_df[features])
    
    # Εύρεση του target player
    target_mask = (same_role_df['Player'] == target_player['Player']) & (same_role_df['Squad'] == target_player['Squad'])
    target_index = same_role_df[target_mask].index[0]
    target_position = same_role_df.index.get_loc(target_index)
    
    # Υπολογισμός Euclidean distances
    distances = euclidean_distances(
        scaled_features[target_position].reshape(1, -1),
        scaled_features
    )[0]
    
    # 🎯 ΒΕΛΤΙΩΜΕΝΗ ΦΟΡΜΟΥΛΑ: Exponential decay
    # Αντί για: (1 / (1 + distance)) * 100
    # Χρησιμοποιούμε: e^(-distance) * 100
    same_role_df['Similarity'] = np.exp(-distances * 1.5) * 100  # 1.5 για λίγο πιο αυστηρό
    
    # Αποκλεισμός του ίδιου παίκτη
    results = same_role_df[same_role_df.index != target_index].sort_values(
        by='Similarity', ascending=False
    )
    
    # Μορφοποίηση
    final = results.head(top_n).copy()
    final['Similarity_Score'] = final['Similarity'].apply(lambda x: f"{x:.1f}%")
    
    return final[['Player', 'Squad', 'Role', 'Gls', 'G/Sh', 'SoT%', 'Similarity_Score']]


# ΕΚΤΕΛΕΣΗ ΜΕ ΒΕΛΤΙΩΜΕΝΗ EUCLIDEAN
print("=" * 60)
print("🔥 ΜΕΘΟΔΟΣ 2: ΒΕΛΤΙΩΜΕΝΗ EUCLIDEAN")
print("=" * 60)

target_player_euc = select_player(df_final)

if target_player_euc is not None:
    print("\n" + "="*60)
    print(f"🔎 ΑΝΑΛΥΣΗ: {target_player_euc['Player']}")
    print(f"🏷️ Ρόλος: {target_player_euc['Role']}")
    print(f"📊 Stats: {target_player_euc['Gls']} Γκολ | {target_player_euc['G/Sh']:.2f} G/Sh")
    print("="*60)
    
    matches_euc = find_similarity_euclidean_improved(df_final, target_player_euc, search_metrics, top_n=10)
    
    if not matches_euc.empty:
        print(f"\n✅ ΟΙ 10 ΚΟΡΥΦΑΙΟΙ ΔΙΑΔΟΧΟΙ (Improved Euclidean):")
        display(matches_euc)

🔥 ΜΕΘΟΔΟΣ 2: ΒΕΛΤΙΩΜΕΝΗ EUCLIDEAN
✅ Βρέθηκε ο παίκτης: Erling Haaland από την ομάδα Manchester City.

🔎 ΑΝΑΛΥΣΗ: Erling Haaland
🏷️ Ρόλος: 💀 Killer Striker
📊 Stats: 20 Γκολ | 0.21 G/Sh

✅ ΟΙ 10 ΚΟΡΥΦΑΙΟΙ ΔΙΑΔΟΧΟΙ (Improved Euclidean):


,Player,Squad,Role,Gls,G/Sh,SoT%,Similarity_Score
5,Ferrán Torres,Barcelona,💀 Killer Striker,11,0.26,53.5,42.2%
9,Robert Lewandowski,Barcelona,💀 Killer Striker,9,0.20,52.5,37.3%
2,Vedat Muriqi,Mallorca,💀 Killer Striker,14,0.17,50.8,36.9%
1,Thiago,Brentford,💀 Killer Striker,16,0.20,57.4,35.0%
28,Christian Pulisic,Milan,💀 Killer Striker,8,0.27,56.7,33.0%
10,Haris Tabakovic,Gladbach,💀 Killer Striker,9,0.24,42.1,32.7%
21,Esteban Lepaul,Rennes,💀 Killer Striker,8,0.18,40.9,32.6%
48,Bradley Barcola,Paris Saint-Germain,💀 Killer Striker,7,0.18,52.5,30.9%
6,Joaquín Panichelli,Strasbourg,💀 Killer Striker,11,0.18,50.0,29.8%
56,Marcus Thuram,Inter,💀 Killer Striker,6,0.15,42.5,28.2%
